In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from reltools.parsers.sql_parser import parse_sql_file
from reltools.parsers.ast_builder import ASTBuilder
from reltools.converters.yaml_converter import ast_to_yaml, save_yaml
from reltools.utils.validators import validate_schema, SchemaValidationError
import yaml

In [3]:
base_dir = Path("/home/basis/work/github/reltools")
input_file = base_dir / 'tests' / 'fixtures' / 'sample_ddl.sql'
output_file = base_dir / 'data' / 'output' / 'demo_output.yaml'

In [4]:
with open(input_file, 'r') as f:
    sql_content = f.read()
    print(sql_content)

-- Sample SQL DDL for testing

CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) NOT NULL
);

CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    user_email VARCHAR(100) NOT NULL,
    order_date DATETIME NOT NULL,
    total DECIMAL(10,2),
    FOREIGN KEY (user_id) REFERENCES users(id)
);



In [18]:
parsed_sql = parse_sql_file(input_file)
parsed_sql

{'tables': [{'name': 'users',
   'columns': [{'name': 'id',
     'type': 'INTEGER',
     'constraints': None,
     'is_primary_key': True},
    {'name': 'username',
     'type': 'VARCHAR(50)',
     'constraints': 'NOT NULL',
     'is_primary_key': False},
    {'name': 'email',
     'type': 'VARCHAR(100)',
     'constraints': 'NOT NULL',
     'is_primary_key': False}],
   'primary_key': ['id'],
   'foreign_keys': []},
  {'name': 'orders',
   'columns': [{'name': 'id',
     'type': 'INTEGER',
     'constraints': None,
     'is_primary_key': True},
    {'name': 'user_id',
     'type': 'INTEGER',
     'constraints': 'NOT NULL',
     'is_primary_key': False},
    {'name': 'user_email',
     'type': 'VARCHAR(100)',
     'constraints': 'NOT NULL',
     'is_primary_key': False},
    {'name': 'order_date',
     'type': 'DATETIME',
     'constraints': 'NOT NULL',
     'is_primary_key': False},
    {'name': 'total',
     'type': 'DECIMAL(10,2)',
     'constraints': None,
     'is_primary_key': Fa

In [8]:
builder = ASTBuilder()
schema = builder.build(parsed_sql)
print(schema)

Schema(tables=[Table(name='users', columns=[Column(name='id', type='INTEGER', constraints=None), Column(name='username', type='VARCHAR(50)', constraints='NOT NULL'), Column(name='email', type='VARCHAR(100)', constraints='NOT NULL')], primary_key=['id'], foreign_keys=[]), Table(name='orders', columns=[Column(name='id', type='INTEGER', constraints=None), Column(name='user_id', type='INTEGER', constraints='NOT NULL'), Column(name='user_email', type='VARCHAR(100)', constraints='NOT NULL'), Column(name='order_date', type='DATETIME', constraints='NOT NULL'), Column(name='total', type='DECIMAL(10,2)', constraints=None)], primary_key=['id'], foreign_keys=[ForeignKey(name='fk_users_user_id', columns=['user_id'], ref_table='users', ref_columns=['id'])])])


In [11]:
yaml_output = ast_to_yaml(schema)
print(yaml_output)

tables:
- name: users
  columns:
  - name: id
    type: INTEGER
  - name: username
    type: VARCHAR(50)
    constraints: NOT NULL
  - name: email
    type: VARCHAR(100)
    constraints: NOT NULL
  primary_key:
  - id
- name: orders
  columns:
  - name: id
    type: INTEGER
  - name: user_id
    type: INTEGER
    constraints: NOT NULL
  - name: user_email
    type: VARCHAR(100)
    constraints: NOT NULL
  - name: order_date
    type: DATETIME
    constraints: NOT NULL
  - name: total
    type: DECIMAL(10,2)
  primary_key:
  - id
  foreign_keys:
  - name: fk_users_user_id
    columns:
    - user_id
    ref_table: users
    ref_columns:
    - id



In [17]:
yaml_output = ast_to_yaml(parsed_sql)
print(yaml_output)

tables:
- name: users
  columns:
  - name: id
    type: INTEGER
    constraints: null
  - name: username
    type: VARCHAR(50)
    constraints: NOT NULL
  - name: email
    type: VARCHAR(100)
    constraints: NOT NULL
  primary_key: []
  foreign_keys: []
- name: orders
  columns:
  - name: id
    type: INTEGER
    constraints: null
  - name: user_id
    type: INTEGER
    constraints: NOT NULL
  - name: user_email
    type: VARCHAR(100)
    constraints: NOT NULL
  - name: order_date
    type: DATETIME
    constraints: NOT NULL
  - name: total
    type: DECIMAL(10,2)
    constraints: null
  primary_key: []
  foreign_keys:
  - name: fk_users_user_id
    columns:
    - user_id
    ref_table: users
    ref_columns:
    - id



In [14]:
# Load YAML file
with open(output_file, 'r') as f:
    schema = yaml.safe_load(f)

In [15]:
schema

{'tables': [{'name': 'users',
   'columns': [{'name': 'id', 'type': 'INTEGER'},
    {'name': 'username', 'type': 'VARCHAR(50)', 'constraints': 'NOT NULL'},
    {'name': 'email', 'type': 'VARCHAR(100)', 'constraints': 'NOT NULL'}],
   'primary_key': ['id']},
  {'name': 'orders',
   'columns': [{'name': 'id', 'type': 'INTEGER'},
    {'name': 'user_id', 'type': 'INTEGER', 'constraints': 'NOT NULL'},
    {'name': 'user_email', 'type': 'VARCHAR(100)', 'constraints': 'NOT NULL'},
    {'name': 'order_date', 'type': 'DATETIME', 'constraints': 'NOT NULL'},
    {'name': 'total', 'type': 'DECIMAL(10,2)'}],
   'primary_key': ['id'],
   'foreign_keys': [{'name': 'fk_users_user_id',
     'columns': ['user_id'],
     'ref_table': 'users',
     'ref_columns': ['id']}]}]}